In [1]:
import os
import sys
import string
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2019,18)
from solution import Explorer, explore_map, split_map
from grid import Grid

In [2]:
test_cases = [
    {
        "map": """#########
#b.A.@.a#
#########""",
        "steps": 8
    },
    {
        "map": """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################""",
        "steps": 86
    },
    {
        "map": """########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################""",
        "steps": 132
    },
    {
        "map": """#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################""",
        "steps": 136
    },
    {
        "map": """########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################""",
        "steps": 81
    }

]

test_cases_2 = [
    {
        "map": """#######
#a.#Cd#
##@#@##
#######
##@#@##
#cB#Ab#
#######""",
        "steps": 8
    },
    {
        "map": """###############
#d.ABC.#.....a#
######@#@######
###############
######@#@######
#b.....#.....c#
###############""",
        "steps": 24
    },
    {
        "map": """#############
#DcBa.#.GhKl#
#.###@#@#I###
#e#d#####j#k#
###C#@#@###J#
#fEbA.#.FgHi#
#############""",
        "steps": 32
    },
    {
        "map": """#############
#g#f.D#..h#l#
#F###e#E###.#
#dCba@#@BcIJ#
#############
#nK.L@#@G...#
#M###N#H###.#
#o#m..#i#jk.#
#############""",
        "steps": 72
    }

]

In [3]:
for idx, test_case in enumerate(test_cases):
    # print(test_case['map'])
    result = explore_map(test_case['map'])
    if result == test_case['steps']:
        print(f"{idx}: passed {result} == {test_case['steps']}")
    else:
        print(f"{idx}: failed {result} == {test_case['steps']}")
    assert result == test_case['steps']

0: passed 8 == 8
1: passed 86 == 86
2: passed 132 == 132
3: passed 136 == 136
4: passed 81 == 81


In [4]:
       
for idx, test_case in enumerate(test_cases_2):
    result = explore_map(test_case['map'])
    if result == test_case['steps']:
        print(f"{idx}: passed {result} == {test_case['steps']}")
    else:
        print(f"{idx}: failed {result} == {test_case['steps']}")

0: passed 8 == 8
1: passed 24 == 24
2: passed 32 == 32
3: passed 72 == 72


In [5]:


key_to_bit = {key: 1 << (ord(key) - ord('a')) for key in string.ascii_lowercase}
print(key_to_bit)

{'a': 1, 'b': 2, 'c': 4, 'd': 8, 'e': 16, 'f': 32, 'g': 64, 'h': 128, 'i': 256, 'j': 512, 'k': 1024, 'l': 2048, 'm': 4096, 'n': 8192, 'o': 16384, 'p': 32768, 'q': 65536, 'r': 131072, 's': 262144, 't': 524288, 'u': 1048576, 'v': 2097152, 'w': 4194304, 'x': 8388608, 'y': 16777216, 'z': 33554432}


In [6]:
test='ac'
test_sum = sum(key_to_bit[key] for key in test)
print(test_sum)
print(f"{test_sum:b}")
test_sum |= key_to_bit['b']
print(f"{test_sum:b}")
test_sum |= key_to_bit['b']
print(f"{test_sum:b}")
print(bin(test_sum).count('1'))

5
101
111
111
3


In [7]:
import string 
string.ascii_lowercase[0]

'a'

In [8]:
a = set([(0,0), (0,1)])
a.update([(2,1)])
print(a)


{(0, 1), (2, 1), (0, 0)}


In [9]:
from __future__ import print_function, division
import collections
import itertools
import fileinput
import heapq

# Kind of slow, takes 7s to run :/


grid = [s.rstrip() for s in fileinput.input('input.txt')]

lingrid = list(itertools.chain.from_iterable(grid))
w, h = len(grid[0]), len(grid)
n = lingrid.index('@')
x, y = n % w, n // w
allkeys = set(c for c in lingrid if c.islower())

grid[y-1] = grid[y-1][:x]   +  '#'  + grid[y-1][x+1:]
grid[  y] = grid[y  ][:x-1] + '###' + grid[y  ][x+2:]
grid[y+1] = grid[y+1][:x]   +  '#'  + grid[y+1][x+1:]
#print('\n'.join(grid))

pos = (
  (x-1, y-1),
  (x+1, y-1),
  (x-1, y+1),
  (x+1, y+1),
)


def reachable_keys(sx, sy, keys):
    q = collections.deque([(sx, sy, 0)])
    #print(q)
    seen = set()
    d = ( (-1, 0), (1, 0), (0, -1), (0, 1) )
    while q:
        cx, cy, l = q.popleft()
        if grid[cy][cx].islower() and grid[cy][cx] not in keys:
            yield l, cx, cy, grid[cy][cx]
            continue
        for dx, dy in d:
            nx, ny = cx + dx, cy + dy
            if ((nx, ny)) in seen:
                continue
            seen.add((nx, ny))

            c = grid[ny][nx]
            if c != '#' and (not c.isupper() or c.lower() in keys):
                q.append((nx, ny, l + 1))


# Shortest path from node "no keys" to node "all keys".
q = [(0, pos, frozenset())]
seen = set()
while q:
    #print(q)
    #print(q[0])
    d, cpos, keys = heapq.heappop(q)
    #print(sorted(keys))
    if keys == allkeys:
        print(d)
        break

    if (cpos, keys) in seen:
        continue
    seen.add((cpos, keys))

    for i, (cx, cy) in enumerate(cpos):
        for l, nx, ny, key in reachable_keys(cx, cy, keys):
            npos = cpos[0:i] + ((nx, ny),) + cpos[i+1:]
            heapq.heappush(q, (d + l, npos, keys | frozenset([key])))


1514


In [10]:
a = [1,2,3]
b = list(reversed(a))
print(a,b)

[1, 2, 3] [3, 2, 1]


In [11]:
a = {(1): 1, (2):2}
key_list = list(a.keys())
for key in key_list:
    a.pop(key)
print(a)

{}
